In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import psycopg2
from bs4 import BeautifulSoup as bp
import time

In [ ]:
import getpass
import os

postgres_url = getpass.getpass("Enter your postgresql url: ")

In [ ]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'


In [ ]:
options=Options()
options.add_argument(f"user-agent={USER_AGENT}")
options.add_argument(f"accept-language=vi-VN,vi;q=0.9")
driver=webdriver.Chrome(options=options)

In [ ]:
def get_hotel_id(name):
    query="""
        SELECT hotel_id
        FROM hotel
        WHERE name ILIKE %s;"""
    conn = psycopg2.connect(postgres_url)
    cur = conn.cursor()
    cur.execute("""SET search_path TO travel_database, public;""")
    cur.execute(query,(name,))
    row=cur.fetchone()
    hotel_id = row[0]
    cur.close()
    return hotel_id

In [ ]:
def insert_db_hotel_price(price,id):
    print(f"insert price into db of room {id}")
    conn = psycopg2.connect(postgres_url)
    cur = conn.cursor()
    cur.execute("""SET search_path TO travel_database, public;""")
    query="""
    UPDATE Hotelprice SET price = %s
    WHERE hotel_id = %s;"""
    cur.execute(query, (price,id))
    conn.commit()
    cur.close()
    conn.close()

In [ ]:
def get_list_id():
    print(f'Taking list of hotel IDs that do not have a price...')
    try:
        conn = psycopg2.connect(postgres_url)
        cur = conn.cursor()
        cur.execute("SET search_path TO travel_database, public;")
        
        query = """
        SELECT hotel_id FROM hotelprice WHERE price IS NULL;
        """
        cur.execute(query)
        hotel_ids = [row[0] for row in cur.fetchall()]
        print(f"Found {len(hotel_ids)} places")
        
    except Exception as e:
        print(f"Error: {e}")
    
    finally:
        cur.close()
        conn.close()
    
    return hotel_ids

In [ ]:
def retake_price(soup,id_list):
    #lay tung o 
    columns=soup.findAll('div',class_='css-1dbjc4n r-14lw9ot r-awg2lu r-1dzdj1l r-rs99b7 r-18u37iz r-1jbys1g r-1udh08x')
    #lay tung ten va gia 
    for column in enumerate(columns):
        name=column.find('h3',class_='css-4rbku5 css-901oao css-cens5h r-a5wbuh r-ubezar r-1kfrs79 r-rjixqe r-fdjqy7')
        if name:
            name=name.text.strip()
            id=get_hotel_id(name)
            if id in id_list:
                price=columns.find('div',class_='')
                price_text = price.get_text().strip()
        #convert to INT
                price_int=int(price_text.replace('VND','').replace('.',''))
                insert_db_hotel_price(price_int,id)
        else:
            print(f'cannot find name for rooms in url')
            break
    


In [ ]:
if __name__ == "__main__":
    list_id=get_list_id()
    for x in range(1,138):
        driver.get(f'https://www.traveloka.com/vi-vn/hotel/vietnam/region/hanoi-10009843/{x}')
        time.sleep(10)
        soup=bp(driver.page_source,'lxml')
        retake_price(soup,list_id)
        
    